In [3]:
import numpy as np 

In [20]:
# Naive Bayes Classifier Class 
class NBC:
    def __init__(self, feature_types, num_classes):
        self.real = [i for i, j in enumerate(feature_types) if j == "r"]
        self.cat = [i for i, j in enumerate(feature_types) if j == "b"]
        self.num_classes = num_classes
    
    def fitGaussian(self, X): 
        variance = np.var(X, axis=1)
        variance[np.argmax(variance <= 0)] = 1e-6
        return np.mean(X, axis=1), variance
    
    def fitBernoulli(self, X): 
        # Define smoothing 
        a = 0.01
        counts = np.count_nonzero(X,axis=1)
        return (counts + a) / (X.shape[1] + (a * 2))
        
    def fit(self, X, y): 
        targClasses, count = np.unique(y, return_counts=True)
        classParamsReal = []
        classParamsCat = []
        for targClass in targClasses:
            classY = np.argwhere(y == targClass)
            realX = np.array(X[self.real, classY])
            catX = np.array(X[self.cat, classY])
            classParamsReal.append(self.fitGaussian(realX))
            classParamsCat.append(self.fitBernoulli(catX))
        self.targClasses = targClasses 
        self.classParamsReal = classParamsReal
        self.classParamsCat = classParamsCat
        self.pis = count / y.size

In [23]:
X = np.array([[1.0,3.0,3.0],[5,2,4],[1.0,2.0,3.0],[1,0,0]])
nbc = NBC(['r','b','r','b'],2)
nbc.fit(np.array([[1,1,1],[1,0,1]]),[1,2,3,1,1])

IndexError: index 2 is out of bounds for axis 0 with size 2